We have a total of 9 excel files and I'm going to import them all together for cleanup

In [3]:
from google.colab import drive
drive.mount('/content/drive')


!pip install pandas openpyxl


import pandas as pd
import os
import glob

def extract_esg_scores(file_path):


    company = os.path.basename(file_path).split(" for ")[-1].split(".xlsx")[0]


    all_sheets = pd.read_excel(file_path, sheet_name=None, header=None)


    all_data = pd.DataFrame()

    for sheet_name, df_sheet in all_sheets.items():

        target_row = df_sheet[df_sheet[1].str.contains("ESG Score \(Weight 100\.0%\)", na=False)]

        if not target_row.empty:

            years = df_sheet.iloc[5, 2::2].values
            values = target_row.iloc[0, 3::2].values


            temp_df = pd.DataFrame({
                'Year': years,
                'Sheet': sheet_name,
                'ESG_Score_Value': values,
                'Company': company
            })

            all_data = pd.concat([all_data, temp_df], ignore_index=True)


    all_data['ESG_Score_Value'] = pd.to_numeric(all_data['ESG_Score_Value'], errors='coerce')
    all_data.dropna(subset=['ESG_Score_Value'], inplace=True)

    return all_data


folder_path = '/content/drive/MyDrive/data_clean_ESG_python/The_raw_data/'
file_pattern = os.path.join(folder_path, 'ESG Table for *.xlsx')


file_paths = glob.glob(file_pattern)


combined_data = pd.DataFrame()


for file_path in file_paths:
    print(f"Processing: {file_path}")
    df = extract_esg_scores(file_path)
    combined_data = pd.concat([combined_data, df], ignore_index=True)


combined_data.sort_values(by=['Company', 'Year'], inplace=True)

output_path = '/content/drive/MyDrive/Combined_ESG_Scores.xlsx'
combined_data.to_excel(output_path, index=False)

print(f"Combined ESG scores saved to {output_path}")

Mounted at /content/drive
Processing: /content/drive/MyDrive/data_clean_ESG_python/The_raw_data/ESG Table for WOW.AX.xlsx
Processing: /content/drive/MyDrive/data_clean_ESG_python/The_raw_data/ESG Table for AD.AS(Ahold Delhaize).xlsx
Processing: /content/drive/MyDrive/data_clean_ESG_python/The_raw_data/ESG Table for SBRY(sainsbury).L.xlsx
Processing: /content/drive/MyDrive/data_clean_ESG_python/The_raw_data/ESG Table for COL.AX(coles).xlsx
Processing: /content/drive/MyDrive/data_clean_ESG_python/The_raw_data/ESG Table for COST.O.xlsx
Processing: /content/drive/MyDrive/data_clean_ESG_python/The_raw_data/ESG Table for L.TO(loblaw.xlsx
Processing: /content/drive/MyDrive/data_clean_ESG_python/The_raw_data/ESG Table for TSCO(Tesco).L.xlsx
Processing: /content/drive/MyDrive/data_clean_ESG_python/The_raw_data/ESG Table for CARR.PA.xlsx
Processing: /content/drive/MyDrive/data_clean_ESG_python/The_raw_data/ESG Table for WMT.xlsx
处理完成！合并后的数据已保存至: /content/drive/MyDrive/Combined_ESG_Scores.xlsx


I cleaned the data from nine excel files and consolidated them into one excel file.

I saved it in Google drive.

After looking at it I realised that the data still needed to be cleaned so I saved it as a csv file and cleaned it again.

In [7]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Combined_ESG_Scores.xlsx - Sheet1.csv'

df = pd.read_csv(file_path)

df['Year'] = pd.to_datetime(df['Year']).dt.year


df_cleaned = df.drop_duplicates(subset=['Year', 'Company'])


pivot_table = df_cleaned.pivot_table(
    index='Year',
    columns='Company',
    values='ESG_Score_Value',
    aggfunc='first'
)


pivot_table.reset_index(inplace=True)
pivot_table.columns.name = None


pivot_table.sort_values(by='Year', inplace=True)


print(pivot_table.head())


pivot_table.to_excel("ESG_Score_Yearly_Table.xlsx", index=False)


pivot_table = df_cleaned.pivot_table(
    index='Year',
    columns='Company',
    values='ESG_Score_Value',
    aggfunc='first'
)
pivot_table.to_excel('/content/drive/MyDrive/ESG_Score_Yearly_Table.xlsx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   Year  AD.AS(Ahold Delhaize)  CARR.PA  COL.AX(coles)  COST.O  L.TO(loblaw  \
0  2014                  86.02    65.04            NaN     NaN          NaN   
1  2015                    NaN    66.40            NaN   69.13        67.61   
2  2016                  68.01    71.93            NaN   56.71        62.35   
3  2017                  74.47    78.79            NaN   60.94        57.18   
4  2018                  77.13    79.53            NaN   66.10        58.42   

   SBRY(sainsbury).L  TSCO(Tesco).L    WMT  WOW.AX  
0              74.95          75.03  77.80     NaN  
1              64.99          74.54  78.84   73.77  
2              65.62          79.24  78.12   71.11  
3              52.66          78.36  75.07   80.35  
4              58.14          73.58  72.33   85.43  
